In [9]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os
import pandas as pd

def old_biner(o_bin,m_bin,f_bin,f_step,l_step):
    a=np.arange(o_bin,m_bin,f_step)
    b=np.arange(m_bin,f_bin+l_step,l_step)
    return np.concatenate((a,b))

def biner(edges,bin_widths,histogram):
    if (len(edges)+1!=len(bin_widths)):
        print("Check edges and bin widths array sizes!")
        return
    
    bins=[]
    first_bin = histogram.GetXaxis().GetBinLowEdge(1)
    last_bin = histogram.GetXaxis().GetBinUpEdge(histogram.GetNbinsX())
    for i in range(0,len(edges)):
        n_spaces = int((edges[i] - first_bin)/bin_widths[i])
        bins = np.concatenate((bins,np.linspace(first_bin,edges[i],n_spaces,endpoint=False)))
        first_bin = edges[i]
        if edges[i]==edges[-1]:
            n_spaces = int((last_bin - edges[i])/bin_widths[i+1])
            bins = np.concatenate((bins,np.linspace(edges[i],last_bin,n_spaces,endpoint=False)))
            bins = np.concatenate((bins,[last_bin]))
    return bins            

def normalization(hist_list,norm_bin):
    for hist in hist_list:
        for i in range(1,hist.GetNbinsX()+1):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=hist.GetBinWidth(i)/norm_bin
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)

In [12]:
##### SELECTING GENERATOR FOR SIGNAL SAMPLE
signal_sample="Signal_Sherpa.root"
ztt_sample="Ztautau_MGRW.root"
fit = True
if fit:
    type_of_mc = ztt_sample.strip('.root').split('_')[1]+'After_Fit'
else :
    type_of_mc = ztt_sample.strip('.root').split('_')[1]
    
vbf_scale = 1.173
qcd_scale = 1.042
#### CREATING THE CORRESPONDING MC.ROOT FILE EVERYTIME YOU CHANGE THE SIGNAL SAMPLE

os.system("rm BG.root MC.root")
os.system('hadd BG.root VV.root singletop.root Wjets.root ttbar.root Zjets.root Higgs.root '+ztt_sample)
os.system("hadd MC.root "+"BG.root "+signal_sample)

0

hadd Target file: BG.root
hadd compression setting for all output: 1
hadd Source file 1: VV.root
hadd Source file 2: singletop.root
hadd Source file 3: Wjets.root
hadd Source file 4: ttbar.root
hadd Source file 5: Zjets.root
hadd Source file 6: Higgs.root
hadd Source file 7: Ztautau_MGRW.root
hadd Target path: BG.root:/
hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_Sherpa.root
hadd Target path: MC.root:/


In [13]:
histos_no_rebin={
"n_bjets":['N b-jets'],
"lepiso":['Lepton Isolation'],
"n_jets_interval":['N jets gap'],
}

histos_rebin = {
"tau_pt":[[25.0,100.0,150.0],[25.0,25.0,50.0,350.0],25.0,'pT(#tau)'],
"lep_pt":[[27,102,150],[27,25,48,350],20,'pT(l)'],
"delta_phi":[[1.8],[0.3,0.7],0.3,'#Delta#phi(#tau,l)'],
"lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#eta(l)'],
"tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#eta(#tau)'],
"delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#DeltaR(#tau,l)'],
"delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#DeltaR(l,j)'],
"delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[[0.5],[0.5,0.5],0.5,'#DeltaR(#tau,j)'],
"met_basic_dphi":[[100],[20,100],20,'MET'],
"delta_y":[[2.0,6.0],[2.0,1.0,4.0],1.0,'#Deltay_{jj}'],
"omega":[[-0.2,1.6],[2.8,0.3,1.4],0.3,'#Omega'],
"rnn_score_1p":[[0.25],[0.25,0.25],0.25,'jetRNN Score 1p'],
"rnn_score_3p":[[0.4],[0.2,0.1999],0.2,'jetRNN Score 3p'],
"ljet0_pt":[[75,400,600],[75,65,100,400],50,'pT(j_{1})'],
"ljet1_pt":[[70,150,350],[70,40,100,650],50,'pT(j_{2})'],
"pt_bal":[[0.15],[0.03,0.75],0.03,'pT balance'],
"Z_centrality":[[0.5],[0.1,0.5],0.1,'#xi(Z)'],
"mass_jj":[[1500,3000],[250,500,1000],250,'m_{jj}'],
"reco_mass_i":[[40,65,115,175],[40,25,10,15,65],10,'m_{#tau,l}(i)'],
"reco_mass_o":[[40,65,115,175],[40,25,10,15,65],10,'m_{#tau,l}(o)'],
"reco_mass_":[[66,116,176],[66,10,20,64],10,'m_{#tau,l}'],
"Z_pt_reco_i_basic_cuts_tpt":[[300],[50,100],50,'pT(Z)']
}

histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)
    
    
for i in histos:
    print(i)
    ROOT.TH1.SetDefaultSumw2()
    MC=ROOT.TFile.Open("MC.root","READ")
    Data = ROOT.TFile.Open("Data.root","READ")
    Ztautau=ROOT.TFile.Open(ztt_sample,"READ")
    VV=ROOT.TFile.Open("VV.root","READ")
    Wjets=ROOT.TFile.Open("Wjets.root","READ")
    Zjets=ROOT.TFile.Open("Zjets.root","READ")
    Ttbar=ROOT.TFile.Open("ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("singletop.root","READ")
    Higgs=ROOT.TFile.Open("Higgs.root","READ")
    Signal=ROOT.TFile.Open(signal_sample,"READ")

    a=i
    
    mc=MC.Get(a)
    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    higgs=Higgs.Get(a)
    ztt=Ztautau.Get(a)
    
    mc.SetDirectory(0)
    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    higgs.SetDirectory(0)
    ztt.SetDirectory(0)
    
    MC.Close()
    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    Higgs.Close()
    Ztautau.Close()

    hs = ROOT.THStack("hs","")
    s=data.GetXaxis().GetBinLowEdge(1)
    e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
    if i in histos_rebin:
        ob=data.GetNbinsX()
        s=data.GetXaxis().GetBinLowEdge(1)
        e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
        rebining=biner(histos[i][0],histos[i][1],data)
        print(rebining)
        nb=len(rebining)-1
        mc=mc.Rebin(nb,"signal",rebining)
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        higgs=higgs.Rebin(nb,"higgs",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        ztt=ztt.Rebin(nb,"ztt",rebining)
        
        hist_list=[signal,data,vv,tt,t,z,w,mc,ztt,
                   higgs]
        normalization(hist_list,histos[i][2])
    
    signal.SetFillColor(ROOT.kOrange+7)
    signal.SetLineWidth(0)
    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan)
    higgs.SetFillColor(ROOT.kWhite)
    higgs.SetLineColor(ROOT.kWhite)
    data.SetLineColor(ROOT.kBlack)
    ztt.SetLineColor(ROOT.kViolet)
    ztt.SetFillColor(ROOT.kViolet)
    data.SetMarkerStyle(20)
    data.SetMarkerSize(0.5)
    
    #################### SCALING FACTORS FROM FIT ####################
    if fit:
        signal.Scale(vbf_scale)
        ztt.Scale(qcd_scale)
            
    ##################################################################

    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(higgs)
    hs.Add(ztt)
    hs.Add(signal)
    
    monte_carlo=signal.Clone()
    monte_carlo.SetLineWidth(1)
    monte_carlo.Add(z,1)
    monte_carlo.Add(tt,1)
    monte_carlo.Add(vv,1)
    monte_carlo.Add(w,1)
    monte_carlo.Add(t,1)
    monte_carlo.Add(higgs,1)
    monte_carlo.Add(ztt,1)
    
    ratio=monte_carlo.Clone()
    ratio.Divide(data)
    
    ratio_sg_mc=signal.Clone()
    ratio_sg_mc.Divide(mc)
    
    gStyle.SetOptStat(1111111)
    gStyle.SetStatY(0.97);                
    gStyle.SetStatX(0.97);
    gStyle.SetStatW(0.12);                
    gStyle.SetStatH(0.12);

    canvas3 = ROOT.TCanvas("canvas3")
    canvas3.cd()

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.03)
    pad1.SetRightMargin(0.03)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()


    data.Draw("pe")
    hs.Draw("HIST same")
    data.Draw("pe same")
    #if "rnn_score" in i:
    pad1.SetLogy()
    data.Draw("sameaxis")
    
    if i=="reco_mass_":
        s=66
        e=116


    data . GetYaxis (). SetRangeUser (0.1,16*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (s,e)
    if i in histos_rebin:
        data . GetYaxis (). SetTitle ("Events/"+str(histos[i][2])+" GeV")
    
        
    legend = ROOT . TLegend (0.45 ,0.80 ,0.85 ,0.95)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jets ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")
    legend . AddEntry ( higgs ," Higgs ")
    legend . AddEntry ( ztt ," Ztt")
    legend.SetNColumns(3)
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (0)
    legend . Draw ()
    

    hs.SetTitle(a)

    data.SetTitle("")
    l=ROOT.TLatex()
    l.SetNDC ()
    l.DrawLatex(0.9,0.7,"Z#rightarrow #taul")
    
    max_ratio = ratio.GetMaximum()
    if max_ratio > 3:
        max_ratio = 3
    if max_ratio < 1.5:
        max_ratio = 1.5
        
    min_ratio = ratio.GetMinimum()
    if min_ratio > 0.7:
        min_ratio = 0.5

    canvas3.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.17 ,1 ,0.35)
    pad2.SetRightMargin(0.03)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.0)
    pad2.Draw ()
    pad2.cd ()
    ratio.SetFillColorAlpha(ROOT.kBlue,0.35)
    ratio.Draw ("E2")
    ratio.SetTitle("")
    ratio.SetStats(0)
    ratio . GetYaxis (). SetRangeUser (min_ratio ,max_ratio)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.15)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    #ratio . GetXaxis (). SetTitle (histos[i][3])
    ratio . GetXaxis (). SetTitleSize (0.00)
    ratio.GetXaxis().SetLabelSize(0.00)
    ratio.GetYaxis().SetLabelSize(0.08)
    ratio.SetMarkerStyle(8)
    ratio.SetMarkerSize(0.6)

        

    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    
    separators = []
    resolution = 10 # In percentage
    i=0
    range_sep = []
    while i < (int(max_ratio)+1):
        step = resolution/100
        if i > min_ratio:
            range_sep.append(i)
        i = i + step
     
    for i in range (len(range_sep)):
        sep = ROOT.TLine(s ,range_sep[i],e,range_sep[i])
        sep.SetLineColor(ROOT.kBlack)
        sep.SetLineWidth(1)
        sep.SetLineStyle(2)
        separators.append(sep)
        
    line . Draw (" same ")   
    for i in separators:
        i.Draw("same")
    
    canvas3.cd()
    pad3 = ROOT . TPad (" pad3","pad3" ,0 ,0.00 ,1 ,0.17)
    pad3.SetRightMargin(0.03)
    pad3.SetLeftMargin(0.08)
    pad3.SetTopMargin(0)
    pad3.SetBottomMargin(0.3)
    pad3.Draw ()
    pad3.cd ()
    ratio_sg_mc.SetStats(0)
    ratio_sg_mc . GetYaxis (). SetRangeUser (0.0 ,1.02)
    ratio_sg_mc . GetXaxis (). SetRangeUser (s ,e)
    ratio_sg_mc . GetYaxis (). SetTitle ("SIGNAL/MC")
    ratio_sg_mc . GetYaxis (). SetTitleSize (0.15)
    ratio_sg_mc . GetYaxis (). SetTitleOffset (0.25)
    ############ X AXIS TITLE #################
    axisTitle='hola'
    if len(histos[a])>2:
        try :
            axisTitle=histos[a][3]    
        except :
            pass
    else :
        try :
            axisTitle=histos[a][0]    
        except :
            pass
    ##########################################
    ratio_sg_mc.SetXTitle(axisTitle)
    ratio_sg_mc.SetTitleSize(0.17,"X")
    ratio_sg_mc.SetTitleOffset(0.8,"X")
    ratio_sg_mc.GetYaxis().SetLabelSize(0.09)
    ratio_sg_mc.GetXaxis().SetLabelSize(0.16)
    ratio_sg_mc.SetMarkerStyle(8)
    ratio_sg_mc.SetMarkerSize(0.6)
    
    ratio_sg_mc.Draw ("hist p")
    
    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line11 = ROOT . TLine (s ,0.80 ,e,0.80)
    line11 . SetLineColor ( ROOT . kBlack )
    line11 . SetLineWidth (1)
    line11 . SetLineStyle(2)
    line12 = ROOT . TLine (s ,0.60 ,e,0.60)
    line12 . SetLineColor ( ROOT . kBlack )
    line12 . SetLineWidth (1)
    line12 . SetLineStyle(2)
    line13 = ROOT . TLine (s ,0.40 ,e,0.40)
    line13 . SetLineColor ( ROOT . kBlack )
    line13 . SetLineWidth (1)
    line13 . SetLineStyle(2)
    line14 = ROOT . TLine (s ,0.20 ,e,0.20)
    line14 . SetLineColor ( ROOT . kBlack )
    line14 . SetLineWidth (1)
    line14 . SetLineStyle(2)
    sline = ROOT . TLine (s ,1 ,e,1)
    sline . SetLineColor ( ROOT . kBlack )
    sline . SetLineWidth (2)
    sline.Draw("same")
    line11 . Draw (" same ")
    line12 . Draw (" same ")
    line13 . Draw (" same ")
    line14 . Draw (" same ")
        







    canvas3.Print(a+"_"+type_of_mc+".pdf")
    
    
    if a=="tau_nprongs_cuts_tpt":
        mc_yield_1p=mc.Integral(1,2)
        data_yield_1p=data.Integral(1,2)
        print("\n"*2)
        print("MC 1p= ",round(mc_yield_1p,3))
        print("DATA 1p= ",round(data_yield_1p,3))
        print("\n"*2)
        mc_yield_3p=mc.Integral(3,4)
        data_yield_3p=data.Integral(3,4)
        print("\n"*2)
        print("MC 3p= ",round(mc_yield_3p,3))
        print("DATA 3p= ",round(data_yield_3p,3))
        print("\n"*2)

n_bjets
lepiso
n_jets_interval
tau_pt
[  0.  25.  50.  75. 100. 150. 500.]
lep_pt
[  0.  27.  52.  77. 102. 150. 500.]
delta_phi
[0.  0.3 0.6 0.9 1.2 1.5 1.8 2.5 3.2]
lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
[-2.5 -2.  -1.5 -1.  -0.5  0.   0.5  1.   1.5  2.   2.5]
tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
[-2.5 -2.  -1.5 -1.  -0.5  0.   0.5  1.   1.5  2.   2.5]
delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6. ]
delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6. ]
delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6. ]
met_basic_dphi
[  0.  20.  40.  60.  80. 100. 200. 300. 400. 500.]
delta_y
[ 0.  2.  3.  4.  5.  6. 10.]
o

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file n_bjets_MGRWAfter_Fit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lepiso_MGRWAfter_Fit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file n_jets_interval_MGRWAfter_Fit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file tau_pt_MGRWAfter_Fit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep_pt_MGRWAfter_Fit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file delta_phi_MGRWAfter_Fit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
